# Odds dataframe

# Remember to change the date of the dataframe!

In [1]:
import pandas as pd
import numpy as np
import requests
import json
import odds

In [2]:
odds_response = requests.get(f'https://api.the-odds-api.com/v4/sports/baseball_mlb/odds', params={
    'api_key': odds.api_key,
    'regions': 'us',
    'markets': 'h2h,spreads',
    'oddsFormat': 'american',
    'dateFormat': 'iso',
})

if odds_response.status_code != 200:
    print(f'Failed to get odds: status_code {odds_response.status_code}, response body {odds_response.text}')

else:
    odds_json = odds_response.json()
    print('Number of events:', len(odds_json))

    # Check the usage quota
    print('Remaining requests', odds_response.headers['x-requests-remaining'])
    print('Used requests', odds_response.headers['x-requests-used'])

Number of events: 14
Remaining requests 491
Used requests 9


In [3]:
def json_to_df(json):
    # This function takes in nested json and outputs a dataframe with the wanted columns
    return pd.json_normalize(json, record_path = ['bookmakers', 'markets', 'outcomes'], meta = ['home_team', 'away_team', ['bookmakers', 'title']])

In [4]:
df = json_to_df(odds_json)

In [5]:
df.head(20)

,name,price,point,home_team,away_team,bookmakers.title
0,Cincinnati Reds,166,NaN,Cincinnati Reds,Philadelphia Phillies,BetOnline.ag
1,Philadelphia Phillies,-182,NaN,Cincinnati Reds,Philadelphia Phillies,BetOnline.ag
2,Cincinnati Reds,100,1.5,Cincinnati Reds,Philadelphia Phillies,BetOnline.ag
3,Philadelphia Phillies,-120,-1.5,Cincinnati Reds,Philadelphia Phillies,BetOnline.ag
4,Cincinnati Reds,155,NaN,Cincinnati Reds,Philadelphia Phillies,DraftKings
5,Philadelphia Phillies,-180,NaN,Cincinnati Reds,Philadelphia Phillies,DraftKings
6,Cincinnati Reds,100,1.5,Cincinnati Reds,Philadelphia Phillies,DraftKings
7,Philadelphia Phillies,-120,-1.5,Cincinnati Reds,Philadelphia Phillies,DraftKings
8,Cincinnati Reds,166,NaN,Cincinnati Reds,Philadelphia Phillies,LowVig.ag
9,Philadelphia Phillies,-180,NaN,Cincinnati Reds,Philadelphia Phillies,LowVig.ag


In [6]:
df = df[df['point'].isnull()]

In [7]:
df.head()

,name,price,point,home_team,away_team,bookmakers.title
0,Cincinnati Reds,166,NaN,Cincinnati Reds,Philadelphia Phillies,BetOnline.ag
1,Philadelphia Phillies,-182,NaN,Cincinnati Reds,Philadelphia Phillies,BetOnline.ag
4,Cincinnati Reds,155,NaN,Cincinnati Reds,Philadelphia Phillies,DraftKings
5,Philadelphia Phillies,-180,NaN,Cincinnati Reds,Philadelphia Phillies,DraftKings
8,Cincinnati Reds,166,NaN,Cincinnati Reds,Philadelphia Phillies,LowVig.ag


In [8]:
df.to_csv('../data/odds_aug_15th.csv', index = False)

In [9]:
max_line = df.loc[df.groupby('name')['price'].idxmax()]

In [10]:
max_line.drop(columns = 'point', inplace = True)
max_line['name'] = max_line['name'].map({'Arizona Diamondbacks':'ARI', 'Atlanta Braves':'ATL', 'Baltimore Orioles':'BAL', 'Boston Red Sox':'BOS', 'Chicago Cubs':'CHC', 'Chicago White Sox':'CHW', 'Cincinnati Reds':'CIN', 'Cleveland Guardians':'CLE', 'Colorado Rockies':'COL', 'Detroit Tigers':'DET', 'Miami Marlins':'MIA', 'Houston Astros':'HOU', 'Kansas City Royals':'KCR', 'Los Angeles Angels':'LAA', 'Los Angeles Dodgers':'LAD', 'Milwaukee Brewers':'MIL', 'Minnesota Twins':'MIN', 'New York Mets':'NYM', 'New York Yankees':'NYY', 'Oakland Athletics':'OAK', 'Philadelphia Phillies':'PHI', 'Pittsburgh Pirates':'PIT', 'San Diego Padres':'SDP', 'San Francisco Giants':'SFG', 'Seattle Mariners':'SEA', 'St. Louis Cardinals':'STL', 'Tampa Bay Rays':'TBR', 'Texas Rangers':'TEX', 'Toronto Blue Jays':'TOR', 'Washington Nationals':'WSN'})

In [11]:
max_line

,name,price,home_team,away_team,bookmakers.title
964,ARI,102,San Francisco Giants,Arizona Diamondbacks,Circa Sports
528,ATL,-141,Atlanta Braves,New York Mets,LowVig.ag
386,BAL,200,Toronto Blue Jays,Baltimore Orioles,FOX Bet
164,BOS,-133,Pittsburgh Pirates,Boston Red Sox,BetUS
222,CHC,-155,Washington Nationals,Chicago Cubs,DraftKings
736,CHW,115,Chicago White Sox,Houston Astros,BetUS
0,CIN,166,Cincinnati Reds,Philadelphia Phillies,BetOnline.ag
470,CLE,-197,Cleveland Guardians,Detroit Tigers,Circa Sports
688,COL,205,St. Louis Cardinals,Colorado Rockies,WynnBET
487,DET,190,Cleveland Guardians,Detroit Tigers,BetRivers


In [12]:
preds = pd.read_csv('../data/preds_aug_16th.csv')
preds.head()

,team,runs_scored,wp,ml
0,CIN,4.157534,0.248379,302.609781
1,PHI,7.232314,0.751621,-302.609781
2,MIA,3.936608,0.475544,110.285647
3,SDP,4.134107,0.524456,-110.285647
4,WSN,3.912342,0.404423,147.265656


In [15]:
compare = preds.merge(max_line, how = 'left', left_on = 'team', right_on = 'name')
compare

,team,runs_scored,wp,ml,name,price,home_team,away_team,bookmakers.title
0,CIN,4.157534,0.248379,302.609781,CIN,166.0,Cincinnati Reds,Philadelphia Phillies,BetOnline.ag
1,PHI,7.232314,0.751621,-302.609781,PHI,-175.0,Cincinnati Reds,Philadelphia Phillies,Intertops
2,MIA,3.936608,0.475544,110.285647,MIA,141.0,Miami Marlins,San Diego Padres,Circa Sports
3,SDP,4.134107,0.524456,-110.285647,SDP,-145.0,Miami Marlins,San Diego Padres,BetUS
4,WSN,3.912342,0.404423,147.265656,WSN,146.0,Washington Nationals,Chicago Cubs,BetOnline.ag
5,CHC,4.747747,0.595577,-147.265656,CHC,-155.0,Washington Nationals,Chicago Cubs,DraftKings
6,PIT,3.700384,0.394519,153.472953,PIT,128.0,Pittsburgh Pirates,Boston Red Sox,Intertops
7,BOS,4.584191,0.605481,-153.472953,BOS,-133.0,Pittsburgh Pirates,Boston Red Sox,BetUS
8,NYY,4.293325,0.559801,-127.170061,NYY,-154.0,New York Yankees,Tampa Bay Rays,WynnBET
9,TBR,3.807162,0.440199,127.170061,TBR,150.0,New York Yankees,Tampa Bay Rays,BetOnline.ag


In [21]:
def ml_diff(ml, price):
    ml_norm = 0
    price_norm = 0
    if ml > 0:
        ml_norm = ml - 100
        if price > 0:
            price_norm = price - 100
        elif price < 0:
            price_norm = price + 100
    elif ml < 0:
        ml_norm = ml + 100
        if price < 0:
            price_norm = price + 100
        elif price > 0:
            price_norm = price - 100
    return price_norm - ml_norm

In [22]:
compare['comparison'] = compare.apply(lambda x: ml_diff(x.ml, x.price), axis=1)

In [28]:
compare.sort_values(by = 'comparison', ascending = False)

,team,runs_scored,wp,ml,name,price,home_team,away_team,bookmakers.title,comparison
1,PHI,7.232314,0.751621,-302.609781,PHI,-175.0,Cincinnati Reds,Philadelphia Phillies,Intertops,127.609781
19,COL,4.603040,0.455037,119.762366,COL,205.0,St. Louis Cardinals,Colorado Rockies,WynnBET,85.237634
11,BAL,4.014532,0.420830,137.625591,BAL,200.0,Toronto Blue Jays,Baltimore Orioles,FOX Bet,62.374409
17,KCR,4.064527,0.451252,121.605579,KCR,173.0,Minnesota Twins,Kansas City Royals,BetOnline.ag,51.394421
25,LAD,5.154088,0.580386,-138.314355,LAD,110.0,Milwaukee Brewers,Los Angeles Dodgers,DraftKings,48.314355
20,TEX,4.463422,0.595707,-147.345367,NaN,NaN,NaN,NaN,NaN,47.345367
26,LAA,3.718564,0.449698,122.371411,LAA,156.0,Los Angeles Angels,Seattle Mariners,BetOnline.ag,33.628589
2,MIA,3.936608,0.475544,110.285647,MIA,141.0,Miami Marlins,San Diego Padres,Circa Sports,30.714353
15,NYM,4.808717,0.488332,104.778523,NYM,133.0,Atlanta Braves,New York Mets,SugarHouse,28.221477
9,TBR,3.807162,0.440199,127.170061,TBR,150.0,New York Yankees,Tampa Bay Rays,BetOnline.ag,22.829939


In [29]:
compare.to_csv('../data/compare_aug_16th.csv', index = False)